Vanilla RNN (using Tensorflow)
--------------

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/airquality_aam_aca/data/kolkata.csv', parse_dates=True)
df.set_index('date')
print(df.head())

In [ ]:
from utility import sequence_to_table
from numpy import mean

# Time series conversion and transformation
seq_mean = mean(df.PM25.values)
sequence = df.PM25.values - seq_mean
data = sequence_to_table(sequence, look_back=30)

# divide data into feature and target
X = data.drop(columns=['next']).values
y = data.next.values

# first reshape the data
X_reshaped = X.reshape((X.shape[0], 1, X.shape[1]))

Vanilla RNN
----------

In [ ]:
# Vanilla RNN model

import tensorflow as tf
from keras.models import Sequential
tf.random.set_seed(42)

model = Sequential(name="Vanilla_RNN")
model.add(tf.keras.layers.SimpleRNN(units=64, 
                              activation='relu', 
                              input_shape=(1, X_reshaped.shape[2])))
model.add(tf.keras.layers.Dense(1))

model.compile(loss='mse', optimizer='adam')
model.summary()

In [ ]:
history = model.fit(X_reshaped, y, epochs=100, batch_size=32)

In [ ]:
# Plot the loss
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 3)).set_dpi(128)
plt.plot(history.history['loss'], label='Loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.show()

In [ ]:
# Model evaluation
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

forecast = model.predict(X_reshaped)
rmse = round(sqrt(mean_squared_error(y, forecast)), 2)
r2 = round(r2_score(y, forecast), 2)

print('RMSE:', rmse)
print('R2 score:', r2)

In [ ]:
plt.figure(figsize=(18, 6)).set_dpi(128)
plt.plot(y[-1000:], label='Actual series')
plt.plot(forecast[-1000:], label='Fitted series')
plt.legend()
plt.xlabel('Days')
plt.ylabel('PM2.5 Concentration Level')
plt.show()

In [ ]:
import numpy as np

predictions = []
input = X_reshaped[-1]

for i in range(60):
  input = input.reshape(1, 1, 30)
  predict = model.predict(input)
  input = input.ravel()
  predictions.append(round(predict[0][0] + seq_mean, 2))
  input = np.delete(input, 0)
  input = np.append(input, predict[0][0])

In [ ]:
for i in predictions:
  print(i)